# Transitions for a two level system with an electron

In [14]:
import numpy as np
import itertools
from operator import add
from functools import reduce

#from itertools import combinations, permutations

We have two sites $(i,j)$ and two levels $\alpha, \beta$. The Initial states are $d_i^1d_j^1$ configurations where one electron sits onthe $\alpha$ level on each site ($4$ in total). The final states are $d_i^0d_j^2$ configurations ($5$ in total).

The problem now is how to represent the states. We take an array where the first 4 elements correspond to the $i$ site and then the first two correspond to the $+$ and $-$ spin $\alpha$ level. Thus the state with spin up on both sites at the $\alpha$ level is [1 0 0 0 1 0 0 0].

As an example, we take the final configuration where in there is on spin up in each level at site $j$ [0 0 0 0 1 0 1 0]

In [2]:
initial = np.array((1, 0, 0, 0, 1, 0, 0, 0))
final = np.array((0, 0, 0, 0, 1, 0, 1, 0))

The question is, is there a transition matrix element between those states? The answer is yes, since the receiving "state" is empty and no spin flip is involved. We can codify this by taking the XOR operation between the initial and final states and checking the positions where we get True. If both are odd or even the hop is allowed, whereas if one is in odd and the other in even positions it is not allowd (implying a spin flip)

In [3]:
jump = np.logical_xor(initial,final)
print(jump.astype(int))

[1 0 0 0 0 0 1 0]


We can see that we get a $1$ in position $0$ and another one in position $6$, thus the jump is allowed since both are even. Let's codify that computationally.

In [4]:
np.nonzero(jump)

(array([0, 6], dtype=int64),)

Now we apply modulo 2, which will allow us to check validity. If both are even or odd, there will be just one unique element.

In [5]:
def allowed(jump):
    if np.unique(np.remainder(np.nonzero(jump),2)).size == 1:
        return 1
    return 0

Testing it out...

In [6]:
allowed(jump)

1

What about another final state like [0 0 0 0 0 1 1 0]. Between the initial and this final state there is a spin flip wich should not be allowed.

In [7]:
final = [0, 0, 0, 0, 0, 1, 1, 0]
jump = np.logical_xor(initial,final)
allowed(jump)

0

As we expected, it is not allowed.

## How to generate the states

The problem now is how to generate the initial and final states. First we start with the initial states. Here we can proceed by isolating one site and writing both possible initial states. Then we double it and make all possible combinations and filter out repeated sequences.

In [8]:
single_initial_states = [[1, 0, 0, 0], [0, 1, 0, 0]]
single_initial_states *= 2
single_initial_states

[[1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0]]

In [9]:
initial_states = set()
for combination in itertools.combinations(single_initial_states,2):
    initial_states.add(tuple(reduce(add, combination)))
list(initial_states)

[(0, 1, 0, 0, 0, 1, 0, 0),
 (1, 0, 0, 0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0, 1, 0, 0),
 (0, 1, 0, 0, 1, 0, 0, 0)]

In [28]:
initial_states = set([tuple(reduce(add, combination)) 
                  for combination 
                  in itertools.combinations(single_initial_states,2)])
initial_states

{(0, 1, 0, 0, 0, 1, 0, 0),
 (0, 1, 0, 0, 1, 0, 0, 0),
 (1, 0, 0, 0, 0, 1, 0, 0),
 (1, 0, 0, 0, 1, 0, 0, 0)}

Now that we have all possible initial states, we can test what happens with our test final state.

In [29]:
for initial_state in initial_states:
    jump = np.logical_xor(initial_state,final)
    print(f"From initial state {initial_state} to final state {final} allowed? {allowed(jump)}")

From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state [0, 0, 0, 0, 0, 1, 1, 0] allowed? 0
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to final state [0, 0, 0, 0, 0, 1, 1, 0] allowed? 0
From initial state (1, 0, 0, 0, 0, 1, 0, 0) to final state [0, 0, 0, 0, 0, 1, 1, 0] allowed? 1
From initial state (0, 1, 0, 0, 1, 0, 0, 0) to final state [0, 0, 0, 0, 0, 1, 1, 0] allowed? 0


To generate the final states, we do simmilarly as for the initial ones. We have to create all the combinations of two elec trons on site $j$. There are in total $6$ states where the one with two electrons in the $\beta$ level is not accessible with optics.

In [27]:
single_states_j = set(itertools.permutations([1, 1, 0, 0]))
final_states = [tuple(reduce(add, [(0, 0, 0, 0), j_state])) 
                for j_state 
                in list(set(itertools.permutations((1, 1, 0, 0))))]
final_states

[(0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 1, 1, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 0, 1, 1, 0),
 (0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 1)]

Having the final states created, we can again test every initial state against every final state.

In [31]:
for initial_state in initial_states:
    for final_state in final_states:
        jump = np.logical_xor(initial_state,final_state)
        print(f"From initial state {initial_state} to final state {final_state} allowed? {allowed(jump)}")

From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 1, 0, 1, 0) allowed? 0
From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 1, 1, 0, 0) allowed? 0
From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 1, 0, 0, 1) allowed? 0
From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 0, 1, 1, 0) allowed? 0
From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 0, 1, 0, 1) allowed? 1
From initial state (0, 1, 0, 0, 0, 1, 0, 0) to final state (0, 0, 0, 0, 0, 0, 1, 1) allowed? 0
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to final state (0, 0, 0, 0, 1, 0, 1, 0) allowed? 1
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to final state (0, 0, 0, 0, 1, 1, 0, 0) allowed? 0
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to final state (0, 0, 0, 0, 1, 0, 0, 1) allowed? 0
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to final state (0, 0, 0, 0, 0, 1, 1, 0) allowed? 0
From initial state (1, 0, 0, 0, 1, 0, 0, 0) to fin